In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.matlib import repmat
import pickle
from sklearn.externals import joblib
import os
import scipy as sp
from scipy.spatial.distance import pdist, squareform
from update_gpparam import *
from tvar_multivariate_G import *
from sample_variances import *
from preprocess_initialization import *
from gpparam_score_diffLogPrior_withProdTerm import *
from calibrator import *

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20

In [2]:
data_path = os.path.join('/Users/xliu/Documents/MRC/Work/Program/','emulator/python_version/emulation_python_ver/')
train_inputs  = data_path + 'LHCDesign_training.txt'
train_outputs = data_path + 'Outputs_training.txt'
valid_inputs  = data_path + 'LHCDesign_validation.txt'
valid_outputs = data_path + 'Outputs_validation.txt'
field_data    = data_path + 'andre_agg_estimates_London_python.txt'	

train_inputs  = read_data_from_txt(train_inputs, is_output = False)
train_outputs = read_data_from_txt(train_outputs, is_output = True, time_length = 245)
valid_inputs  = read_data_from_txt(valid_inputs, is_output = False)
valid_outputs = read_data_from_txt(valid_outputs, is_output = True, time_length = 245)
field_data    = read_data_from_txt(field_data, is_output=False)	

data = {'train_inputs': train_inputs, 'train_outputs': train_outputs,
        'valid_inputs': valid_inputs, 'valid_outputs': valid_outputs}

In [3]:
from emulator import emulator_cls

In [ ]:
emulator = emulator_cls(data, time_length = 245)
emulator.fit()
model_name = 'emulator.sav'
joblib.dump(emulator, data_path+model_name)

In [4]:
# load model
model_name = 'emulator.sav'
emulator_built = joblib.load(data_path + model_name)
print(emulator_built.emulator_param.shape)
#param_posterior = emulator_built.calibrate(field_data)

(8, 245, 1200)


In [6]:
param_t = 0.4 * np.ones((1, 6))
param_extend = repmat(param_t, m = 200, n = 1)

In [7]:
train_inputs = np.random.randn(200, 6)

In [8]:
beta_t = 0.5 * np.ones((1, 6))
beta_extend = repmat(beta_t, m = 200, n = 1)
beta_extend.shape

(200, 6)

In [9]:
beta_t.shape

(1, 6)

In [10]:
para_beta = np.c_[param_extend, beta_extend]

In [11]:
print(para_beta.shape)
para_beta[0, :]

(200, 12)


array([ 0.4,  0.4,  0.4,  0.4,  0.4,  0.4,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5])

In [12]:
list(map(lambda para, x, b: (b*np.sqrt(((para-x)**2))).sum(), param_extend, train_inputs, beta_t))

[2.0108357105585366]

In [13]:
list(map(lambda para, x: (para[6:]*np.sqrt(((para[:6]-x)**2))).sum(), para_beta, train_inputs))

[2.0108357105585366,
 2.3526723434846746,
 2.3579728925307171,
 3.1414083851570238,
 0.84363929494829693,
 2.8407132040344765,
 2.7327836649942641,
 1.5436680995358909,
 1.7934125675297046,
 3.4711100350031323,
 2.5174355345388939,
 1.4871709601659751,
 2.4359697723494405,
 3.626763103066819,
 2.4262226228837096,
 3.953778116826542,
 1.9874498892552694,
 3.0540137372272467,
 0.53189467022827674,
 2.2330399725415879,
 1.8457251143878892,
 2.294378137963375,
 3.1279721115599117,
 1.4962732732082746,
 1.4608305355753539,
 3.8141383024251927,
 1.8027612156508075,
 2.4475318895372746,
 1.9852253017078556,
 3.3542657217739142,
 3.3884229119656233,
 1.9073259820672317,
 2.768710288374197,
 1.645319481146909,
 4.282224170458873,
 2.251218678803153,
 2.7061313263377258,
 1.7030678857600696,
 2.7038535554401886,
 2.939137098157703,
 3.8531818693864803,
 2.3173292221955948,
 2.1653638927878656,
 2.8738938138487211,
 2.6663068336416131,
 2.0483670390113997,
 2.1609903328374562,
 2.1669569674111298

In [2]:
delay_distribution  = np.array([0.00356498122310219, 0.0241909650774591,\
            0.0537030705484224,  0.07956071762201, 0.0961914714596036,\
            0.102950833972866, 0.101591781393376,  0.0946065162134661,\
            0.0843575216352006, 0.0727223345103803, 0.061022715024489,\
            0.050087644508563, 0.0403632249361386, 0.0320249177166795])

In [5]:
delay_distribution

array([ 0.00356498,  0.02419097,  0.05370307,  0.07956072,  0.09619147,
        0.10295083,  0.10159178,  0.09460652,  0.08435752,  0.07272233,
        0.06102272,  0.05008764,  0.04036322,  0.03202492])

In [4]:
np.cumsum(delay_distribution)

array([ 0.00356498,  0.02775595,  0.08145902,  0.16101973,  0.25721121,
        0.36016204,  0.46175382,  0.55636034,  0.64071786,  0.71344019,
        0.77446291,  0.82455055,  0.86491378,  0.8969387 ])

In [8]:
np.flip(np.cumsum(delay_distribution), axis=0)

array([ 0.8969387 ,  0.86491378,  0.82455055,  0.77446291,  0.71344019,
        0.64071786,  0.55636034,  0.46175382,  0.36016204,  0.25721121,
        0.16101973,  0.08145902,  0.02775595,  0.00356498])

In [9]:
from scipy.stats import gamma

In [11]:
a = gamma.mean(0.01, scale = 0.01)
a

0.0001

In [12]:
a = gamma.mean(0.01, scale = 100)
a

1.0

In [6]:
for t in range(0, 10):
    if t in [1, 2, 3]:
        print('days 2, 3 or 4')
    else:
        print(t)

0
days 2, 3 or 4
days 2, 3 or 4
days 2, 3 or 4
4
5
6
7
8
9
